In [471]:
import sys
sys.path.insert(0,"libs")

In [474]:
import pdfplumber
import re

def extract_paragraphs_from_pdf(pdf_path):
    paragraphs = []

    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text = page.extract_text()
            paragraphs.append(text)

    return paragraphs

pdf_path = 'mcq_questions.pdf'
paragraphs = extract_paragraphs_from_pdf(pdf_path)
answer = paragraphs[-1]
patterNumber =  r'(\d+\.\s)'
patternA  = r'(?=\b[a-d]\.\s)'

paragraph = []
start_indices =[]
end_indices = []

complete_paragraphs = "".join(paragraphs)
complete_paragraphs = complete_paragraphs.split("Answers")
complete_paragraphs.pop()
complete_paragraphs = "".join(complete_paragraphs)


for pr in re.finditer(patterNumber, complete_paragraphs):
    start_index = pr.start()
    end_index = pr.end()
    start_indices.append(start_index)
    end_indices.append(end_index)

for i in range(1,len(start_indices)-1):
    matched_strng = complete_paragraphs[start_indices[i-1]:start_indices[i]]
    paragraph.append(matched_strng)

paragraph = [re.split(patternA,p) for p in paragraph]


valid = [p for p in paragraph if len(p) == 5 ]
invalid = [p for p in paragraph if len(p) != 5]

answer_parts = answer.split("Answers")
all_answer = answer_parts[-1].split("\n***")[0]
all_answer = all_answer.replace("\n"," ")
splits :list[str] = all_answer.split(" ")
splits = [ s for s in splits if len(s.strip()) != 0 ]

all_question = []

for v in valid:
    questions : str = v[0].split(".")
    for s in splits:
        s:str
        number = s.split(".")
        if(questions[0].strip() == number[0].strip()):
            correct = 0,
            if number[1] == "a" : correct = 0
            elif number[1] == "b" : correct = 1
            elif number[1] == "c" : correct = 2
            elif number[1] == "d" : correct = 3
            else : correct = 0
            print(
                questions[1].strip()
            )
            all_question.append([
                questions[1].strip(),
                v[1].replace("a.","").strip(),
                v[2].replace("b.","").strip(),
                v[3].replace("c.","").strip(),
                v[4].replace("d.","").strip(),
                correct,
                1,
                ""
            ])



Which structure will perform better during earthquake?
Which of the following structural loads are not applied commonly to a building?
The deformation per unit length is called
The property of a material by which it can be beaten or rolled into thin plates is called
The compression test is carried on
The moment of inertia of a body is always minimum with respect to its
Moment of inertia is the
Sagging, the bending moment occurs at the
Why is base plate provided in short roof trusses?
Which law is also called as the elasticity law?
Whenever a material is loaded within elastic limit, stress is
The ratio of linear stress to the linear strain is called
The unit of modulus of elasticity is same as those of
When a change in length takes place, the strain is known as
The change in length due to a tensile or compressive force acting on a body is given by
P
If we use link support in a structural system, then how many unknowns would we have?
Young’s modulus may be defined as the ratio of
Modulus

In [473]:
import csv

file_path = "mcq_questions.csv"

with open(file_path,mode="w",newline= "\n") as file:
    writer = csv.writer(file)
    writer.writerow(["Question", "answer", "answer", "answer", "answer", "correct", "marks", "description"])
    writer.writerows(all_question)